# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 13

Group Members:
- Jonathan Wiguna (18222019)
- Naomi Pricilla Agustine(18222065)
- Harry Truman Suhalim (18222081)
- Micky Valentino(18222093)

## Import Libraries

In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
from urllib.parse import urlparse
from sklearn.decomposition import PCA

## Import Dataset

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?id=1KJsJrbObEun6NRme9jMIjX8SyIeuTY5M')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [228]:
df = df.drop(columns=['FILENAME', 'id'])


In [229]:
# pemisahan fitur yang bertipe kategori dan non-kategori agar dianalisis dengan pendekatan yang berbeda
categorical = ['TLD',  'Title', 'Domain','URL',  'IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon', 'Robots', 'IsResponsive', 'HasDescription', 'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo']

non_categorical = [col for col in df.columns if col not in categorical]

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [230]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...
train_set, val_set = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
train_set

In [ ]:
val_set

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

Identify Missing Values

In [ ]:
# Identifikasi missing values
print("Jumlah missing values:")
print(df.isnull().sum())
print()

# Menghitung persentase missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
print("Persentase missing values:")
print(missing_percent)

Data Imputation

In [ ]:
# Assign url dengan modus ketika tidak ada Domain dan IsHTTPS
if 'URL' in train_set.columns:
    # Mencari url paling sering/modus
    most_frequent_url = train_set['URL'].mode()[0]

    mask_both_missing = pd.isna(train_set['IsHTTPS']) & pd.isna(train_set['Domain'])
    train_set.loc[mask_both_missing & pd.isna(train_set['URL']), 'URL'] = most_frequent_url

    mask_one_exists = ((pd.notna(train_set['IsHTTPS']) & pd.isna(train_set['Domain'])) | 
            (pd.isna(train_set['IsHTTPS']) & pd.notna(train_set['Domain'])))
    train_set.loc[mask_one_exists & pd.isna(train_set['URL']), 'URL'] = most_frequent_url

# Assign Domain dan TLD jika terdapat URL
for index, row in train_set.iterrows():
    if pd.notna(row['URL']):
        try:
            # Cek HTTPS
            train_set.at[index, 'IsHTTPS'] = 1 if row['URL'].startswith('https://') else 0
            
            parsed = urlparse(row['URL'])
            if not parsed.netloc: # Buat domain yang tidak punya protokol
                parsed = urlparse("http://" + row['URL'])
            
            domain = parsed.netloc or parsed.path

            # Extract TLD
            if '.' in domain:
                tld = domain.split('.')[-1]
                
                # Update Domain and TLD 
                if pd.isna(train_set.at[index, 'Domain']):
                    train_set.at[index, 'Domain'] = domain
                if pd.isna(train_set.at[index, 'TLD']):
                    train_set.at[index, 'TLD'] = tld
        except:
            continue
    # Jika URL tidak ada tapi domaion ada
    elif pd.notna(row['Domain']) and pd.isna(row['TLD']):
        domain = row['Domain']
        if '.' in domain:
            train_set.at[index, 'TLD'] = domain.split('.')[-1]
    
    # Isi URL jika Domain dan IsHTTPS ada
    if pd.isna(row['URL']) and pd.notna(row['Domain']) and pd.notna(row['IsHTTPS']):
        protocol = 'https://' if row['IsHTTPS'] == 1 else 'http://'
        train_set.at[index, 'URL'] = f"{protocol}www.{row['Domain']}"

# Proses URL-based features
if 'URL' in train_set.columns:
    # Update URLLength berdasarkan URL yang ada
    mask = train_set['URL'].notna()
    train_set.loc[mask, 'URLLength'] = train_set.loc[mask, 'URL'].str.len()
    
    # Update letter dan digit counts berdasarkan URL yang ada
    train_set.loc[mask, 'NoOfLettersInURL'] = train_set.loc[mask, 'URL'].str.count('[a-zA-Z]')
    train_set.loc[mask, 'NoOfDegitsInURL'] = train_set.loc[mask, 'URL'].str.count('[0-9]')
    train_set.loc[mask, 'NoOfEqualsInURL'] = train_set.loc[mask, 'URL'].str.count('=')
    train_set.loc[mask, 'NoOfQMarkInURL'] = train_set.loc[mask, 'URL'].str.count(r'\?')
    train_set.loc[mask, 'NoOfAmpersandInURL'] = train_set.loc[mask, 'URL'].str.count('&')
    
    url_lengths = train_set.loc[mask, 'URL'].str.len()
    train_set.loc[mask, 'LetterRatioInURL'] = train_set.loc[mask, 'NoOfLettersInURL'] / url_lengths
    train_set.loc[mask, 'DegitRatioInURL'] = train_set.loc[mask, 'NoOfDegitsInURL'] / url_lengths
    
    special_chars_pattern = '[^a-zA-Z0-9=?&]'
    train_set.loc[mask, 'NoOfOtherSpecialCharsInURL'] = train_set.loc[mask, 'URL'].str.count(special_chars_pattern)
    
    # CharContinuationRate
    char_series = train_set.loc[mask, 'URL']
    continuation_rates = []
    
    for url in char_series:
        if len(url) < 2:
            continuation_rates.append(0)
        else:
            continuous = sum(1 for i in range(len(url)-1) if url[i] == url[i+1])
            continuation_rates.append(continuous / (len(url)-1))
    
    train_set.loc[mask, 'CharContinuationRate'] = continuation_rates

# Proses Domain-based features
if 'Domain' in train_set.columns:
    # Update DomainLength berdasarkan Domain yang ada
    mask = train_set['Domain'].notna()
    train_set.loc[mask, 'DomainLength'] = train_set.loc[mask, 'Domain'].str.len()
    
    # Update IsDomainIP berdasarkan Domain yang ada
    train_set.loc[mask, 'IsDomainIP'] = train_set.loc[mask, 'Domain'].apply(
        lambda x: float(bool(re.match(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$', str(x)))))
    
    # Update NoOfSubDomain berdasarkan Domain yang ada
    train_set.loc[mask, 'NoOfSubDomain'] = train_set.loc[mask, 'Domain'].str.count(r'\.')

# Proses TLD-based features
if 'TLD' in train_set.columns:
    mask = train_set['TLD'].notna()
    train_set.loc[mask, 'TLDLength'] = train_set.loc[mask, 'TLD'].str.len()

# Hubungan HasObfuscation dengan NoOfObfuscatedChar
mask = train_set['NoOfObfuscatedChar'] == 0
train_set.loc[mask & train_set['HasObfuscation'].isna(), 'HasObfuscation'] = 0

mask = train_set['NoOfObfuscatedChar'] > 0
train_set.loc[mask & train_set['HasObfuscation'].isna(), 'HasObfuscation'] = 1

# NoOfSelfRedirect tidak boleh lebih besar dari NoOfURLRedirect
mask = train_set['NoOfURLRedirect'].notna() & train_set['NoOfSelfRedirect'].notna()
invalid_mask = mask & (train_set['NoOfSelfRedirect'] > train_set['NoOfURLRedirect'])
train_set.loc[invalid_mask, 'NoOfSelfRedirect'] = train_set.loc[invalid_mask, 'NoOfURLRedirect']

# HasExternalFormSubmit berdasarkan NoOfExternalRef
mask = train_set['NoOfExternalRef'] > 0
train_set.loc[mask & train_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 1

mask = train_set['NoOfExternalRef'] == 0
train_set.loc[mask & train_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 0

# Proses title 
mask = train_set['Title'].notna()
train_set.loc[mask & train_set['HasTitle'].isna(), 'HasTitle'] = 1
train_set.loc[~mask & train_set['HasTitle'].isna(), 'HasTitle'] = 0

# Isi missing values yang tersisa
boolean_cols = ['HasObfuscation', 'IsHTTPS', 'HasTitle',
                'HasFavicon', 'IsResponsive', 'HasDescription',
                'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
                'HasHiddenFields', 'HasPasswordField', 'HasCopyrightInfo']

count_cols = ['NoOfObfuscatedChar', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'NoOfPopup',
              'NoOfiFrame', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
              'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']

# Handle missing values sisanya
for col in boolean_cols:
    if col in train_set.columns and train_set[col].isnull().any():
        train_set[col] = train_set[col].fillna(train_set[col].mode()[0])

for col in count_cols:
    if col in train_set.columns and train_set[col].isnull().any():
        train_set[col] = train_set[col].fillna(train_set[col].median())

# Handle remaining numerical columns
numerical_cols = train_set.select_dtypes(include=['float64', 'int64']).columns
remaining_cols = [col for col in numerical_cols if col not in boolean_cols + count_cols]
for col in remaining_cols:
    if train_set[col].isnull().any():
        train_set[col] = train_set[col].fillna(train_set[col].median())

print("\nJumlah missing values setelah diisi:")
print(train_set.isnull().sum())

In [ ]:
# Assign url dengan modus ketika tidak ada Domain dan IsHTTPS
if 'URL' in val_set.columns:
    # Mencari url paling sering/modus
    most_frequent_url = val_set['URL'].mode()[0]

    mask_both_missing = pd.isna(val_set['IsHTTPS']) & pd.isna(val_set['Domain'])
    val_set.loc[mask_both_missing & pd.isna(val_set['URL']), 'URL'] = most_frequent_url

    mask_one_exists = ((pd.notna(val_set['IsHTTPS']) & pd.isna(val_set['Domain'])) | 
            (pd.isna(val_set['IsHTTPS']) & pd.notna(val_set['Domain'])))
    val_set.loc[mask_one_exists & pd.isna(val_set['URL']), 'URL'] = most_frequent_url

# Assign Domain dan TLD jika terdapat URL
for index, row in val_set.iterrows():
    if pd.notna(row['URL']):
        try:
            # Cek HTTPS
            val_set.at[index, 'IsHTTPS'] = 1 if row['URL'].startswith('https://') else 0
            
            parsed = urlparse(row['URL'])
            if not parsed.netloc: # Buat domain yang tidak punya protokol
                parsed = urlparse("http://" + row['URL'])
            
            domain = parsed.netloc or parsed.path

            # Extract TLD
            if '.' in domain:
                tld = domain.split('.')[-1]
                
                # Update Domain and TLD 
                if pd.isna(val_set.at[index, 'Domain']):
                    val_set.at[index, 'Domain'] = domain
                if pd.isna(val_set.at[index, 'TLD']):
                    val_set.at[index, 'TLD'] = tld
        except:
            continue
    # Jika URL tidak ada tapi domaion ada
    elif pd.notna(row['Domain']) and pd.isna(row['TLD']):
        domain = row['Domain']
        if '.' in domain:
            val_set.at[index, 'TLD'] = domain.split('.')[-1]
    
    # Isi URL jika Domain dan IsHTTPS ada
    if pd.isna(row['URL']) and pd.notna(row['Domain']) and pd.notna(row['IsHTTPS']):
        protocol = 'https://' if row['IsHTTPS'] == 1 else 'http://'
        val_set.at[index, 'URL'] = f"{protocol}www.{row['Domain']}"

# Proses URL-based features
if 'URL' in val_set.columns:
    # Update URLLength berdasarkan URL yang ada
    mask = val_set['URL'].notna()
    val_set.loc[mask, 'URLLength'] = val_set.loc[mask, 'URL'].str.len()
    
    # Update letter dan digit counts berdasarkan URL yang ada
    val_set.loc[mask, 'NoOfLettersInURL'] = val_set.loc[mask, 'URL'].str.count('[a-zA-Z]')
    val_set.loc[mask, 'NoOfDegitsInURL'] = val_set.loc[mask, 'URL'].str.count('[0-9]')
    val_set.loc[mask, 'NoOfEqualsInURL'] = val_set.loc[mask, 'URL'].str.count('=')
    val_set.loc[mask, 'NoOfQMarkInURL'] = val_set.loc[mask, 'URL'].str.count(r'\?')
    val_set.loc[mask, 'NoOfAmpersandInURL'] = val_set.loc[mask, 'URL'].str.count('&')
    
    url_lengths = val_set.loc[mask, 'URL'].str.len()
    val_set.loc[mask, 'LetterRatioInURL'] = val_set.loc[mask, 'NoOfLettersInURL'] / url_lengths
    val_set.loc[mask, 'DegitRatioInURL'] = val_set.loc[mask, 'NoOfDegitsInURL'] / url_lengths
    
    special_chars_pattern = '[^a-zA-Z0-9=?&]'
    val_set.loc[mask, 'NoOfOtherSpecialCharsInURL'] = val_set.loc[mask, 'URL'].str.count(special_chars_pattern)
    
    # CharContinuationRate
    char_series = val_set.loc[mask, 'URL']
    continuation_rates = []
    
    for url in char_series:
        if len(url) < 2:
            continuation_rates.append(0)
        else:
            continuous = sum(1 for i in range(len(url)-1) if url[i] == url[i+1])
            continuation_rates.append(continuous / (len(url)-1))
    
    val_set.loc[mask, 'CharContinuationRate'] = continuation_rates

# Proses Domain-based features
if 'Domain' in val_set.columns:
    # Update DomainLength berdasarkan Domain yang ada
    mask = val_set['Domain'].notna()
    val_set.loc[mask, 'DomainLength'] = val_set.loc[mask, 'Domain'].str.len()
    
    # Update IsDomainIP berdasarkan Domain yang ada
    val_set.loc[mask, 'IsDomainIP'] = val_set.loc[mask, 'Domain'].apply(
        lambda x: float(bool(re.match(r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$', str(x)))))
    
    # Update NoOfSubDomain berdasarkan Domain yang ada
    val_set.loc[mask, 'NoOfSubDomain'] = val_set.loc[mask, 'Domain'].str.count(r'\.')

# Proses TLD-based features
if 'TLD' in val_set.columns:
    mask = val_set['TLD'].notna()
    val_set.loc[mask, 'TLDLength'] = val_set.loc[mask, 'TLD'].str.len()

# Hubungan HasObfuscation dengan NoOfObfuscatedChar
mask = val_set['NoOfObfuscatedChar'] == 0
val_set.loc[mask & val_set['HasObfuscation'].isna(), 'HasObfuscation'] = 0

mask = val_set['NoOfObfuscatedChar'] > 0
val_set.loc[mask & val_set['HasObfuscation'].isna(), 'HasObfuscation'] = 1

# NoOfSelfRedirect tidak boleh lebih besar dari NoOfURLRedirect
mask = val_set['NoOfURLRedirect'].notna() & val_set['NoOfSelfRedirect'].notna()
invalid_mask = mask & (val_set['NoOfSelfRedirect'] > val_set['NoOfURLRedirect'])
val_set.loc[invalid_mask, 'NoOfSelfRedirect'] = val_set.loc[invalid_mask, 'NoOfURLRedirect']

# HasExternalFormSubmit berdasarkan NoOfExternalRef
mask = val_set['NoOfExternalRef'] > 0
val_set.loc[mask & val_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 1

mask = val_set['NoOfExternalRef'] == 0
val_set.loc[mask & val_set['HasExternalFormSubmit'].isna(), 'HasExternalFormSubmit'] = 0

# Proses title 
mask = val_set['Title'].notna()
val_set.loc[mask & val_set['HasTitle'].isna(), 'HasTitle'] = 1
val_set.loc[~mask & val_set['HasTitle'].isna(), 'HasTitle'] = 0

# Isi missing values yang tersisa
boolean_cols = ['IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle',
                'HasFavicon', 'IsResponsive', 'HasDescription',
                'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
                'HasHiddenFields', 'HasPasswordField', 'HasCopyrightInfo']

count_cols = ['NoOfSubDomain', 'NoOfObfuscatedChar', 'NoOfLettersInURL',
              'NoOfDegitsInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL', 
              'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
              'NoOfURLRedirect', 'NoOfSelfRedirect', 'NoOfPopup',
              'NoOfiFrame', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
              'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef']

# Handle missing values sisanya
for col in boolean_cols:
    if col in val_set.columns and val_set[col].isnull().any():
        val_set[col] = val_set[col].fillna(val_set[col].mode()[0])

for col in count_cols:
    if col in val_set.columns and val_set[col].isnull().any():
        val_set[col] = val_set[col].fillna(val_set[col].median())

# Handle remaining numerical columns
numerical_cols = val_set.select_dtypes(include=['float64', 'int64']).columns
remaining_cols = [col for col in numerical_cols if col not in boolean_cols + count_cols]
for col in remaining_cols:
    if val_set[col].isnull().any():
        val_set[col] = val_set[col].fillna(val_set[col].median())

print("\nJumlah missing values setelah diisi:")
print(val_set.isnull().sum())

In [ ]:
df.describe()

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
import pandas as pd
import numpy as np

# Misalkan val_set adalah DataFrame Anda dan non_categorical adalah daftar kolom non-kategorikal
threshold_percentage = 0.03  # Batas maksimal 3% outlier

outlier = True
while outlier:
    change = False
    for col in non_categorical:
        if val_set[col].skew() > 1 or val_set[col].skew() < -1:
            # Menggunakan IQR untuk mendeteksi outlier
            Q1 = val_set[col].quantile(0.25)
            Q3 = val_set[col].quantile(0.75)
            IQR = Q3 - Q1
            # Mendapatkan indeks outlier
            outlier_indices = val_set[(val_set[col] < Q1 - 1.5 * IQR) | (val_set[col] > Q3 + 1.5 * IQR)].index
        else:
            # Menggunakan z-score untuk mendeteksi outlier
            z_scores = np.abs((val_set[col] - val_set[col].mean()) / val_set[col].std())
            outlier_indices = val_set[z_scores > 3].index
        
        # Hitung persentase outlier
        outlier_percentage = len(outlier_indices) / len(val_set)
        if 0 < outlier_percentage < threshold_percentage:
            val_set = val_set.drop(outlier_indices)
            change = True
        elif outlier_percentage > threshold_percentage:
            Q1 = val_set[col].quantile(0.25)
            Q3 = val_set[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            val_set[col] = np.where(val_set[col] < lower_bound, lower_bound, val_set[col])
            val_set[col] = np.where(val_set[col] > upper_bound, upper_bound, val_set[col])

    print (change)
    if not(change):
        outlier = False

    # Reset index setelah penghapusan outlier
    val_set.reset_index(drop=True, inplace=True)


In [ ]:
import pandas as pd
import numpy as np

# Misalkan train_set adalah DataFrame Anda dan non_categorical adalah daftar kolom non-kategorikal
threshold_percentage = 0.03  # Batas maksimal 3% outlier
outlier = True
while outlier:
    change = False
    for col in non_categorical:
        if train_set[col].skew() > 1 or train_set[col].skew() < -1:
            # Menggunakan IQR untuk mendeteksi outlier
            Q1 = train_set[col].quantile(0.25)
            Q3 = train_set[col].quantile(0.75)
            IQR = Q3 - Q1
            # Mendapatkan indeks outlier
            outlier_indices = train_set[(train_set[col] < Q1 - 1.5 * IQR) | (train_set[col] > Q3 + 1.5 * IQR)].index
        else:
            # Menggunakan z-score untuk mendeteksi outlier
            z_scores = np.abs((train_set[col] - train_set[col].mean()) / train_set[col].std())
            outlier_indices = train_set[z_scores > 3].index
        
        # Hitung persentase outlier
        outlier_percentage = len(outlier_indices) / len(train_set)
        if 0 < outlier_percentage < threshold_percentage:
            train_set = train_set.drop(outlier_indices)
            change = True
        elif outlier_percentage > threshold_percentage:
            Q1 = train_set[col].quantile(0.25)
            Q3 = train_set[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            train_set[col] = np.where(train_set[col] < lower_bound, lower_bound, train_set[col])
            train_set[col] = np.where(train_set[col] > upper_bound, upper_bound, train_set[col])

    print (change)
    if not(change):
        outlier = False

    # Reset index setelah penghapusan outlier
    train_set.reset_index(drop=True, inplace=True)


In [ ]:
for col in non_categorical:
    if train_set[col].skew() > 1 or train_set[col].skew() < -1:
        # Menggunakan IQR untuk mendeteksi outlier
        Q1 = train_set[col].quantile(0.25)
        Q3 = train_set[col].quantile(0.75)
        IQR = Q3 - Q1
        # Mendapatkan indeks outlier
        outlier_indices = train_set[(train_set[col] < Q1 - 1.5 * IQR) | (train_set[col] > Q3 + 1.5 * IQR)].index
    else:
        # Menggunakan z-score untuk mendeteksi outlier
        z_scores = np.abs((train_set[col] - train_set[col].mean()) / train_set[col].std())
        outlier_indices = train_set[z_scores > 3].index
    
    # Hitung persentase outlier
    outlier_percentage = len(outlier_indices) / len(train_set) * 100

    print (outlier_percentage)

In [ ]:
for col in non_categorical:
    if val_set[col].skew() > 1 or val_set[col].skew() < -1:
        # Menggunakan IQR untuk mendeteksi outlier
        Q1 = val_set[col].quantile(0.25)
        Q3 = val_set[col].quantile(0.75)
        IQR = Q3 - Q1
        # Mendapatkan indeks outlier
        outlier_indices = val_set[(val_set[col] < Q1 - 1.5 * IQR) | (val_set[col] > Q3 + 1.5 * IQR)].index
    else:
        # Menggunakan z-score untuk mendeteksi outlier
        z_scores = np.abs((val_set[col] - val_set[col].mean()) / val_set[col].std())
        outlier_indices = val_set[z_scores > 3].index
    
    # Hitung persentase outlier
    outlier_percentage = len(outlier_indices) / len(val_set) * 100

    print (outlier_percentage)

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
jumlah_duplikat = train_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

In [ ]:
jumlah_duplikat = val_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

In [243]:
# Write your code here
train_set = train_set.drop_duplicates()

In [244]:
# Write your code here
val_set = val_set.drop_duplicates()

In [ ]:
jumlah_duplikat = train_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

In [ ]:
jumlah_duplikat = val_set.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
print(len(df.columns))

In [ ]:
# Buat fitur baru dari karakteristik URL
if 'URL' in train_set.columns:
   # Hitung karakter spesial
   train_set['special_char_count'] = train_set['URL'].str.count(r'[^a-zA-Z0-9]').fillna(0)
   # Hitung digit
   train_set['digit_count'] = train_set['URL'].str.count(r'[0-9]').fillna(0)

# Buat fitur interaksi
if 'URLLength' in train_set.columns and 'DomainLength' in train_set.columns:
   train_set['url_domain_ratio'] = train_set['URLLength'] / train_set['DomainLength'].replace(0, 1)

# Buat fitur binary dari TLD
if 'TLD' in train_set.columns:
   common_tlds = ['com', 'org', 'net', 'edu']
   train_set['is_common_tld'] = train_set['TLD'].isin(common_tlds).astype(int)

# Buat aggregate features
security_cols = ['HasObfuscation', 'IsHTTPS', 'HasHiddenFields']
train_set['security_score'] = train_set[security_cols].sum(axis=1)

# Cek fitur baru
print("\nJumlah kolom setelah feature engineering:", len(train_set.columns))
print("\nBeberapa baris dengan fitur baru:")
print(train_set[['special_char_count', 'digit_count', 'url_domain_ratio',
          'is_common_tld', 'security_score']].head())

In [ ]:
# Buat fitur baru dari karakteristik URL
if 'URL' in val_set.columns:
   # Hitung karakter spesial
   val_set['special_char_count'] = val_set['URL'].str.count(r'[^a-zA-Z0-9]').fillna(0)
   # Hitung digit
   val_set['digit_count'] = val_set['URL'].str.count(r'[0-9]').fillna(0)

# Buat fitur interaksi
if 'URLLength' in val_set.columns and 'DomainLength' in val_set.columns:
   val_set['url_domain_ratio'] = val_set['URLLength'] / val_set['DomainLength'].replace(0, 1)

# Buat fitur binary dari TLD
if 'TLD' in val_set.columns:
   common_tlds = ['com', 'org', 'net', 'edu']
   val_set['is_common_tld'] = val_set['TLD'].isin(common_tlds).astype(int)

# Buat aggregate features
security_cols = ['HasObfuscation', 'IsHTTPS', 'HasHiddenFields']
val_set['security_score'] = val_set[security_cols].sum(axis=1)

# Cek fitur baru
print("\nJumlah kolom setelah feature engineering:", len(val_set.columns))
print("\nBeberapa baris dengan fitur baru:")
print(val_set[['special_char_count', 'digit_count', 'url_domain_ratio',
          'is_common_tld', 'security_score']].head())

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

Drop Column that Useless

In [250]:
train_set = train_set.drop(columns=['Title', 'Domain', 'URL'])

In [251]:
val_set = val_set.drop(columns=['Title', 'Domain', 'URL'])

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [252]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
# Identifikasi kolom numerik dari train_set yang sudah ada
numeric_cols = train_set.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop('label') if 'label' in numeric_cols else numeric_cols

# Pilih hanya kolom numerik dari train_set dan val_set
train_set_numeric = train_set[numeric_cols]
val_set_numeric = val_set[numeric_cols]

# Standardization
scaler = StandardScaler()
train_set_scaled = scaler.fit_transform(train_set_numeric)
val_set_scaled = scaler.transform(val_set_numeric)

# Convert kembali ke DataFrame
train_set_scaled = pd.DataFrame(train_set_scaled, columns=train_set_numeric.columns)
val_set_scaled = pd.DataFrame(val_set_scaled, columns=val_set_numeric.columns)

# Cek hasil
train_set_scaled.describe()

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [254]:
# Write your code here
# Write your code here
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Inisialisasi OneHotEncoder
# The sparse argument is deprecated in newer versions.
# It's now implicitly handled based on other settings.
# You can remove it safely.
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # If you need to handle_unknown values

# Melakukan fit_transform pada kolom TLD
encoded_data = encoder.fit_transform(train_set[['TLD']])

# Mengubah hasil encoding menjadi DataFrame
encoded_train_set = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['TLD']))

# Menggabungkan hasil encoding dengan DataFrame asli (opsional)
train_set = pd.concat([train_set, encoded_train_set], axis=1)
train_set = train_set.drop(columns=['TLD'])


# Applying the encoding transformation to the validation set
encoded_val_data = encoder.transform(val_set[['TLD']])

# Creating a DataFrame with the encoded features for the validation set
encoded_val_set = pd.DataFrame(encoded_val_data, columns=encoder.get_feature_names_out(['TLD']))

# Concatenating the encoded features with the original validation set (optional)
val_set = pd.concat([val_set, encoded_val_set], axis=1)
val_set = val_set.drop(columns=['TLD'])


In [ ]:
train_set

In [ ]:
val_set

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
print("distribution:")
print(df['label'].value_counts())
print("\npercentage distribution:")
print(df['label'].value_counts(normalize=True) * 100)

In [258]:
# columns_to_drop_imbalanced = ['id', 'FILENAME', 'URL', 'Title', 'Domain']
# X_imbalanced = df.drop(columns_to_drop_imbalanced + ['label'], axis=1)
# X_imbalanced = pd.get_dummies(X_imbalanced, columns=['TLD'])
# y_imbalanced = df['label']

# print("features shape:", X_imbalanced.shape)
# print("label shape:", y_imbalanced.shape)

# X_train_imbalanced, X_test_imbalanced, y_train_imbalanced, y_test_imbalanced = train_test_split(
#     X_imbalanced, 
#     y_imbalanced, 
#     test_size=0.3, 
#     random_state=66
# )

# print("\ntraining set before resampling:")
# print(y_train_imbalanced.value_counts())
# print("\ntest set:")
# print(y_test_imbalanced.value_counts())

# from imblearn.combine import SMOTETomek
# smote_tomek = SMOTETomek(random_state=66)
# X_train_balanced, y_train_balanced = smote_tomek.fit_resample(X_train_imbalanced, y_train_imbalanced)

# print("\ntraining set  after resampling:")
# print(pd.Series(y_train_balanced).value_counts())

### IV. Normalization

In [ ]:
train_set.describe()

In [ ]:
val_set.describe()

In [ ]:
numeric_cols = train_set.select_dtypes(include=['float64', 'int64']).columns

min_max_scaler = MinMaxScaler()
min_max_scaled = min_max_scaler.fit_transform(train_set[numeric_cols])

train_set_min_max_scaled = train_set.copy()
train_set_min_max_scaled[numeric_cols] = min_max_scaled

print("\nMin-Max Scaled Data:")
print(train_set_min_max_scaled)

val_set_min_max_scaled = val_set.copy()
val_set_min_max_scaled[numeric_cols] = min_max_scaler.transform(val_set[numeric_cols])

print("\nMin-Max Scaled Data:")
print(val_set_min_max_scaled)

# Mengganti kolom numerik asli dengan kolom yang sudah di-scaling pada train_set
train_set = train_set.drop(columns=numeric_cols)
train_set = pd.concat([train_set, train_set_min_max_scaled[numeric_cols]], axis=1)

# Mengganti kolom numerik asli dengan kolom yang sudah di-scaling pada val_set
val_set = val_set.drop(columns=numeric_cols)
val_set = pd.concat([val_set, val_set_min_max_scaled[numeric_cols]], axis=1)

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [265]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [266]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [267]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

In [268]:
X_train = train_set.drop('label', axis=1)
y_train = train_set['label']
X_val = val_set.drop('label', axis=1)
y_val = val_set['label']

## A. KNN

In [ ]:
train_set.select_dtypes(include=['object']).info()

In [ ]:
val_set.select_dtypes(include=['object']).info()

In [ ]:
X_train.select_dtypes(include=['object']).info()

In [ ]:
X_train.select_dtypes(include=['object']).info()

### From Scratch

In [273]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def manhattan_distance(x1, x2):
    return np.sum(np.abs(x1 - x2))

def minkowski_distance(x1, x2, p=2):
    return np.power(np.sum(np.power(np.abs(x1 - x2), p)), 1/p)

def calculate_distances(x_test, X_train, metric='euclidean', p=2):
    distances = []
    
    for x_train in X_train:
        if metric == 'euclidean':
            dist = euclidean_distance(x_test, x_train)
        elif metric == 'manhattan':
            dist = manhattan_distance(x_test, x_train)
        elif metric == 'minkowski':
            dist = minkowski_distance(x_test, x_train, p)
        else:
            raise ValueError("Metrik jarak tidak valid. Gunakan 'euclidean', 'manhattan', atau 'minkowski'")
        distances.append(dist)
    
    return np.array(distances)

def get_neighbors(X_train, y_train, x_test, k, metric='euclidean', p=2):
    distances = calculate_distances(x_test, X_train, metric, p)
    k_indices = np.argsort(distances)[:k]
    k_nearest_labels = y_train[k_indices]
    return k_nearest_labels

def get_majority_vote(labels):
    labels = list(labels)
    unique_labels = list(set(labels))
    votes = [labels.count(label) for label in unique_labels]
    return unique_labels[np.argmax(votes)]

def knn_predict(X_train, y_train, X_test, k=3, metric='euclidean', p=2):
    predictions = []
    
    for x in X_test:
        neighbors = get_neighbors(X_train, y_train, x, k, metric, p)
        majority_label = get_majority_vote(neighbors)
        predictions.append(majority_label)
    
    return np.array(predictions)

In [ ]:
predictions = knn_predict(X_train, y_train, X_val, k=3, metric='euclidean')
print(predictions)

### From Scikit-Learn

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_predictions = knn.predict(X_val)

## B. Naive Bayes

### From Scratch

In [ ]:
def calculate_mean_std(X, y):
    classes = np.unique(y)
    mean_std = {}
    
    for c in classes:
        X_c = X[y == c]
        mean_c = np.mean(X_c, axis=0)
        std_c = np.std(X_c, axis=0)
        std_c = np.where(std_c == 0, 1e-6, std_c)
        mean_std[c] = {'mean': mean_c, 'std': std_c}
    
    return mean_std

def calculate_prior(y):
    classes = np.unique(y)
    n_samples = len(y)
    prior = {}
    
    for c in classes:
        prior[c] = np.sum(y == c) / n_samples
    
    return prior

def gaussian_probability(x, mean, std):
    exponent = np.exp(-((x - mean) ** 2) / (2 * std ** 2))
    return exponent / (np.sqrt(2 * np.pi) * std)

def predict_sample(x, mean_std, prior):
    probabilities = {}
    
    for c in prior.keys():
        class_probability = np.log(prior[c])
        mean = mean_std[c]['mean']
        std = mean_std[c]['std']
        
        feature_probabilities = np.sum(np.log(gaussian_probability(x, mean, std)))
        probabilities[c] = class_probability + feature_probabilities
    
    return max(probabilities.items(), key=lambda x: x[1])[0]

def gaussian_naive_bayes_fit(X, y):
    mean_std = calculate_mean_std(X, y)
    prior = calculate_prior(y)
    return mean_std, prior

def gaussian_naive_bayes_predict(X, mean_std, prior):
    y_pred = []
    
    for x in X:
        prediction = predict_sample(x, mean_std, prior)
        y_pred.append(prediction)
    
    return np.array(y_pred)

### From Scikit-Learn

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_predictions = nb.predict(X_val)# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Evaluation Metrics
knn_accuracy = accuracy_score(y_val, knn_predictions)
knn_precision = precision_score(y_val, knn_predictions)
knn_recall = recall_score(y_val, knn_predictions)
knn_f1 = f1_score(y_val, knn_predictions)

nb_accuracy = accuracy_score(y_val, nb_predictions)
nb_precision = precision_score(y_val, nb_predictions)
nb_recall = recall_score(y_val, nb_predictions)
nb_f1 = f1_score(y_val, nb_predictions)

# Print the evaluation metrics
print("KNN Metrics:")
print("Accuracy:", knn_accuracy)
print("Precision:", knn_precision)
print("Recall:", knn_recall)
print("F1 Score:", knn_f1)

print("\nNaive Bayes Metrics:")
print("Accuracy:", nb_accuracy)
print("Precision:", nb_precision)
print("Recall:", nb_recall)
print("F1 Score:", nb_f1)

In [240]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [241]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`